In [1]:
# import ipykernel
import pandas as pd

In [ ]:
# Load data
# Clean data
# Scale data

In [ ]:
# Categorical

# Numerical

In [ ]:
class Preprocessor(dataset):

  def __init__(self, dataset):
    self.dataset = dataset
    self.num_features = num_features
    self.cat_features = cat_features

  def clean(self)
    return self.X, self.y
  def scale(self):
    

  def split(self, test_size):
   X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=test_size, random_state=42)


In [ ]:
class Modeller(dict(X_train, y_train, X_test, y_test)):

  def __init__(self):
    self.X
    self.y
  
  def batch_model(models, X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame):
  """
  models: Dictionary
  X_train, y_train, X_test, y_test: Dataframes
  """

  all_scores = {}
  for model_name, regressor in tqdm(models.items()):

      model = regressor

      # Measure the time it takes to fit the model
      start_time_fit = time.time()
      model.fit(X_train, y_train)
      end_time_fit = time.time()
      fit_time = end_time_fit - start_time_fit

      # Train pred
      y_train_pred = model.predict(X_train)

      # Test pred
      y_test_pred = model.predict(X_test)

      # Adjusted R2
      adjusted_r2_train=adjusted_r2_score(y_train, y_train_pred, X_train)
      adjusted_r2_test=adjusted_r2_score(y_test, y_test_pred, X_test)

      scoring = {
          'mae': mean_absolute_error,
          'mape': mean_absolute_percentage_error,
          'mse': mean_squared_error,
          'rmse': lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
          'r2': r2_score,
      }

      all_scores[model_name] = {
          'fit_time': fit_time,
          'train_adjusted_r2': adjusted_r2_train,
          'test_adjusted_r2': adjusted_r2_test,

      }

      for metric_name, scoring_function in scoring.items():
          # Use the appropriate scoring function to calculate the score on the training set
          train_score = scoring_function(y_train, y_train_pred)
          all_scores[model_name][f'train_{metric_name}'] = train_score

          # Use the appropriate scoring function to calculate the score on the test set
          test_score = scoring_function(y_test, y_test_pred)
          all_scores[model_name][f'test_{metric_name}'] = test_score

  def hypertune(self, model_class, params):
    all_scores={}
    def search_fn(params):
        with mlflow.start_run():
            mlflow.set_tag('person','frankie') # Change name
            mlflow.set_tag('model','xgb')
            mlflow.log_params(params)

            start_time_fit = time.time()
            model=model_class(**params) # Change the model here
            model.fit(X_train_full_cat_int, y_train_full_cat_int)
            end_time_fit = time.time()
            fit_time = end_time_fit - start_time_fit

            # Train pred
            y_train_final_pred = model.predict(X_train_full_cat_int)

            # Test pred
            y_test_final_pred = model.predict(X_test_full_cat_int)

            scoring = {
                'mae': mean_absolute_error,
                'mape': mean_absolute_percentage_error,
                'mse': mean_squared_error,
                'rmse': lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
                'r2': r2_score,
            }

            all_scores['xgb'] = {
                'fit_time': fit_time
            }

            for metric_name, scoring_function in scoring.items():
                # Use the appropriate scoring function to calculate the score on the training set
                train_score = scoring_function(y_train_full_cat_int, y_train_final_pred)
                all_scores['xgb'][f'train_{metric_name}'] = train_score

                # Use the appropriate scoring function to calculate the score on the test set
                test_score = scoring_function(y_test_full_cat_int, y_test_final_pred)
                all_scores['xgb'][f'test_{metric_name}'] = test_score

            scores_df=pd.DataFrame(all_scores.values(), index=all_scores.keys())


            mlflow.log_metrics(all_scores['xgb'])

            return {'loss': all_scores['xgb']['test_mae'], 'status': STATUS_OK}

    search_space = {
        'n_estimators': scope.int(hp.uniform('n_estimators', 100, 500)),
        'max_depth': scope.int(hp.quniform('max_depth', 10, 50, 1)),
        'learning_rate': hp.loguniform('learning_rate', -7, -2), # exp(-3), exp(0)
        'min_child_weight': hp.loguniform('min_child_weight', -1, 5),
        'objective': 'reg:absoluteerror',
        'random_state': 42,
        'booster': 'gbtree'
    }
    trials=Trials()
    best_result = fmin(
        fn=search_fn,
        space=search_space,
        algo=tpe.suggest,
        max_evals=30,
        trials=trials
    )